In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

2024-03-19 18:32:15.064737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 18:32:15.064830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 18:32:15.066529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 18:32:15.076007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 18:32:15.865091: W tensorflow/compiler/tf2

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [5]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [6]:
747/4825

0.15481865284974095

In [7]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [8]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [9]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [10]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(1494, 2)

In [11]:
df_balanced['Category'].value_counts()

Category
spam    747
ham     747
Name: count, dtype: int64

In [12]:
df_balanced.sample(10)

,Category,Message
122,ham,here is my new address -apples&pairs&all that ...
240,spam,U 447801259231 have a secret admirer who is lo...
4227,ham,"Ok thats cool. Its , just off either raglan rd..."
914,ham,Ok lor but not too early. Me still having proj...
4636,ham,How come i din c ü... Yup i cut my hair...
645,ham,Allo! We have braved the buses and taken on th...
330,ham,I'm reading the text i just sent you. Its mean...
2442,ham,Back in brum! Thanks for putting us up and kee...
3111,spam,Good Luck! Draw takes place 28th Feb 06. Good ...
353,ham,TODAY is Sorry day.! If ever i was angry with ...


In [13]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x:1 if x=='spam' else 0)
df_balanced.sample(10)

,Category,Message,spam
2100,spam,"SMS SERVICES. for your inclusive text credits,...",1
2625,ham,S da..al r above &lt;#&gt;,0
1598,spam,URGENT! Your Mobile number has been awarded wi...,1
3379,ham,Hows that watch resizing,0
840,spam,Last chance 2 claim ur £150 worth of discount ...,1
4287,ham,Congrats. That's great. I wanted to tell you n...,0
4308,ham,I guess it is useless calling u 4 something im...,0
3359,ham,I don't run away frm u... I walk slowly &amp; ...,0
4503,ham,Wot is u up 2 then bitch?,0
1504,ham,Ill be there on &lt;#&gt; ok.,0


In [14]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify= df_balanced['spam'])

In [17]:
x_train.head(4)

3856    Free msg: Single? Find a partner in your area!...
4909             I'm in solihull, | do you want anything?
312     Think ur smart ? Win £200 this week in our wee...
4334                Y bishan lei... I tot ü say lavender?
Name: Message, dtype: object

In [26]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2024-03-19 18:58:56.550309: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 18:58:56.834369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 18:58:56.834446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 18:58:56.838380: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 18:58:56.838457: I external/local_xla/xla/stream_executor

In [27]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']
    

get_sentence_embedding([
    "$500 discount. Hurry up !",
    "Bhavin, are you up for a volleyball game tonight ?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.88113064, -0.4790945 , -0.9251654 , ..., -0.78634757,
        -0.7585211 ,  0.9169145 ],
       [-0.8166446 , -0.51383746, -0.9361561 , ..., -0.855867  ,
        -0.68493974,  0.8223356 ]], dtype=float32)>

In [28]:
e = get_sentence_embedding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [29]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.7606918 , -0.14219403,  0.49604598, ...,  0.42165288,
        -0.5322141 ,  0.8031218 ],
       [-0.8602321 , -0.21242923,  0.49156916, ...,  0.3979804 ,
        -0.6050627 ,  0.84471625],
       [-0.7128866 , -0.15463923,  0.38401574, ...,  0.35278666,
        -0.5099136 ,  0.73474145],
       [-0.82533485, -0.35550597, -0.5906968 , ..., -0.01613715,
        -0.61417586,  0.8723028 ],
       [-0.7504135 , -0.2681261 , -0.26689595, ...,  0.02839455,
        -0.5938099 ,  0.7974986 ],
       [-0.785443  , -0.29949614,  0.41027665, ...,  0.52225524,
        -0.49573493,  0.8150743 ]], dtype=float32)>

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]], [e[1]])

array([[0.9911089]], dtype=float32)

In [33]:
cosine_similarity([e[3]], [e[4]])

array([[0.9872034]], dtype=float32)

In [35]:
# BERT Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural Network Layers

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)
# Construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [38]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS)

In [40]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2024-03-19 19:38:08.826135: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f25c82a0f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-19 19:38:08.826220: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-03-19 19:38:08.871976: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-19 19:38:08.958981: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


 1/35 [..............................] - ETA: 6:40 - loss: 0.7596 - accuracy: 0.5312 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1710857289.101401   19499 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 [==============================] - 95s 2s/step - loss: 0.6844 - accuracy: 0.5696 - precision: 0.5728 - recall: 0.5482
Epoch 2/10
35/35 [==============================] - 24s 679ms/step - loss: 0.5351 - accuracy: 0.7884 - precision: 0.7879 - recall: 0.7893
Epoch 3/10
35/35 [==============================] - 24s 672ms/step - loss: 0.4530 - accuracy: 0.8393 - precision: 0.8381 - recall: 0.8411
Epoch 4/10
35/35 [==============================] - 23s 664ms/step - loss: 0.4086 - accuracy: 0.8687 - precision: 0.8668 - recall: 0.8714
Epoch 5/10
35/35 [==============================] - 24s 678ms/step - loss: 0.3742 - accuracy: 0.8830 - precision: 0.8705 - recall: 0.9000
Epoch 6/10
35/35 [==============================] - 24s 694ms/step - loss: 0.3443 - accuracy: 0.8875 - precision: 0.8754 - recall: 0.9036
Epoch 7/10
35/35 [==============================] - 25s 702ms/step - loss: 0.3262 - accuracy: 0.8839 - precision: 0.8881 - recall: 0.8786
Epoch 8/10
35/35 [==============================

In [41]:
model.evaluate(x_test, y_test)

12/12 [==============================] - 9s 641ms/step - loss: 0.2677 - accuracy: 0.9225 - precision: 0.8835 - recall: 0.9733


[0.2677006721496582,
 0.9224599003791809,
 0.8834951519966125,
 0.9732620120048523]

In [42]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

12/12 [==============================] - 8s 666ms/step


In [43]:
import numpy as np


y_predicted = np.where(y_predicted > 0.5, 1,0)
y_predicted

array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,

In [44]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

1/1 [==============================] - 1s 1s/step


array([[0.81930596],
       [0.8746011 ],
       [0.8429422 ],
       [0.2690752 ],
       [0.14450873]], dtype=float32)

Working !